<a href="https://colab.research.google.com/github/Bob0123456/Homework/blob/main/MMPretrain_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

题目：基于 ResNet50 的水果分类

背景：使用基于卷积的深度神经网络 ResNet50 对 30 种水果进行分类

任务

划分训练集和验证集 按照 MMPreTrain CustomDataset 格式组织训练集和验证集 使用 MMPreTrain 算法库，编写配置文件，正确加载预训练模型 在水果数据集上进行微调训练 使用 MMPreTrain 的 ImageClassificationInferencer 接口，对网络水果图像，或自己拍摄的水果图像，使用训练好的模型进行分类 需提交的验证集评估指标（不能低于 60%）

ResNet-50

作业数据集下载： 链接：https://pan.baidu.com/s/1YgoU1M_v7ridtXB9xxbA1Q 提取码：52m9

课程中猫狗数据集下载地址： https://download.openmmlab.com/mmclassification/dataset/cats_dogs_dataset.tar

## 环境配置

In [21]:
!git clone https://github.com/open-mmlab/mmpretrain.git
%cd mmpretrain
!pip install -U openmim && mim install -e .

fatal: destination path 'mmpretrain' already exists and is not an empty directory.
/content/mmpretrain/mmpretrain
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/mmpretrain/mmpretrain
ERROR: file:///content/mmpretrain/mmpretrain does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [22]:
!mim install -e ".[multimodal]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/mmpretrain/mmpretrain
ERROR: file:///content/mmpretrain/mmpretrain does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [23]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 检查是否安装成功

In [24]:
import mmpretrain

print(mmpretrain.__version__)

1.0.0rc8


In [25]:
from mmpretrain import get_model, list_models, inference_model

print("图像分类任务种包含resnet18的模型:", list_models(task='Image Classification', pattern='resnet18'))
print("图像描述任务种包含blip的模型:", list_models(task='Image Caption', pattern='blip'))


图像分类任务种包含resnet18的模型: ['resnet18_8xb16_cifar10', 'resnet18_8xb32_in1k']
图像描述任务种包含blip的模型: ['blip-base_3rdparty_caption', 'blip2-opt2.7b_3rdparty-zeroshot_caption']


In [26]:
!python demo/image_demo.py demo/demo.JPEG resnet18_8xb32_in1k --device gpu

python3: can't open file '/content/mmpretrain/mmpretrain/demo/image_demo.py': [Errno 2] No such file or directory


## 从google云盘下载数据集

In [27]:
# 进入mmpretrain文件夹
%cd /content/mmpretrain

/content/mmpretrain


In [28]:
# 创建data文件夹
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [29]:
#从Google云端硬盘下载数据集fruit30_trian.tar
!gdown https://drive.google.com/u/0/uc?id=16AvoLcKzCfvsKFRVQ-RGVnsTRWzAe7ze&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=16AvoLcKzCfvsKFRVQ-RGVnsTRWzAe7ze
To: /content/mmpretrain/fruit30_train.tar
100% 187M/187M [00:01<00:00, 133MB/s]


In [30]:
#将下载的tar数据集 解压到data所对应的目录下
import tarfile

tar_path = "/content/mmpretrain/fruit30_train.tar"  # 压缩包路径
destination_path = "/content/mmpretrain/data/fruits"  # 目标文件夹的路径

with tarfile.open(tar_path) as tar:
    tar.extractall(path=destination_path)

In [31]:
# 确认数据集类别数
import os

folder_path = "/content/mmpretrain/data/fruits"  # replace with your folder path
items = os.listdir(folder_path)

folder_count = sum([1 for item in items if os.path.isdir(os.path.join(folder_path, item))])

print("Number of folders:", folder_count)

Number of folders: 30


## 数据集划分

In [32]:
import os
import shutil
import random
# File Location
data_path = "/content/mmpretrain/data/fruits"  # 源数据路径
train_path = "./data/train"  # 训练集路径
valid_path = "./data/valid"  # 验证集路径
test_path = "./data/test"  # 测试集路径
 
# 创建train, valid, test文件夹
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
 
# 遍历data文件夹下的每个子文件夹
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        # 创建相应的子文件夹
        os.makedirs(os.path.join(train_path, folder), exist_ok=True)
        os.makedirs(os.path.join(valid_path, folder), exist_ok=True)
        os.makedirs(os.path.join(test_path, folder), exist_ok=True)
 
        # 获取子文件夹中的文件列表
        files = os.listdir(folder_path)
        # 打乱文件顺序！！！！！！！！！！！！！！！
        random.shuffle(files)
 
        # 计算训练、验证、测试集的文件数量
        train_count = int(len(files) * 0.7)
        valid_count = int(len(files) * 0.2)
        test_count = len(files) - train_count - valid_count
 
        # 分别划分训练、验证、测试集
        train_files = files[:train_count]
        valid_files = files[train_count:(train_count+valid_count)]
        test_files = files[(train_count+valid_count):]
 
        # 移动文件
        for file in train_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(train_path, folder, file))
        for file in valid_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(valid_path, folder, file))
        for file in test_files:
            shutil.move(os.path.join(folder_path, file), os.path.join(test_path, folder, file))

# 获取完整配置文件

In [33]:
!python /content/mmpretrain/tools/train.py /content/mmpretrain/configs/resnet/resnetv1d50_8xb32_in1k.py

06/08 02:05:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 1015997484
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50

In [34]:
# !python /content/mmpretrain/tools/train.py /content/mmpretrain/configs/convmixer/convmixer-768-32_10xb64_in1k.py

## 修改配置文件

In [37]:
cfg="""
# 模型配置文件
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNetV1d',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=30,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, 5),
    ),
    init_cfg=dict(type='Pretrained', checkpoint='https://download.openmmlab.com/mmclassification/v0/resnet/resnetv1d50_b32x8_imagenet_20210531-db14775a.pth'))
# dataset settings
dataset_type = 'CustomDataset'
data_preprocessor = dict(
    num_classes=30,
    # RGB format normalization parameters
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    to_rgb=True,
)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', scale=224, backend='pillow'),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackInputs'),
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='ResizeEdge', scale=256, edge='short', backend='pillow'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackInputs'),
]

train_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='/content/mmpretrain/data/train',
        pipeline=train_pipeline),
    sampler=dict(type='DefaultSampler', shuffle=True),
)

val_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='/content/mmpretrain/data/valid',
        pipeline=test_pipeline),
    sampler=dict(type='DefaultSampler', shuffle=False),
)
val_evaluator = dict(type='Accuracy', topk=1)

# If you want standard test, please manually configure the test dataset
test_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type=dataset_type,
        data_root='/content/mmpretrain/data/test',
        pipeline=test_pipeline),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

test_evaluator = val_evaluator

# optimizer 训练优化的流程(优化器)
optim_wrapper = dict(
    optimizer=dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001))

# learning policy 参数规划器，在训练期间修改学习率之类的参数
param_scheduler = dict(
    type='MultiStepLR', by_epoch=True, milestones=[30, 60, 90], gamma=0.1)

# train, val, test setting 流程设置
#最大100个epoch，每个epoch训练完成后都进行一次验证
train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict() # 空字典表示使用默认配置，不配置额外的参数
test_cfg = dict()

# NOTE: `auto_scale_lr` is for automatically scaling LR,
# based on the actual training batch size.
# 一般batch size越小，使用越小的学习率
# 和学习率有关，会根据batchsize缩放学习率，
# 训练resnet是用8张卡训练，每张卡的batch_size是32，和数据集的配置一致为32
auto_scale_lr = dict(base_batch_size=256)

# 运行参数配置
# defaults to use registries in mmpretrain
default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.日志配置
    # interval表示间隔多少次迭代打印一次日志
    logger=dict(type='LoggerHook', interval=10),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.权重保存配置
    # interval=1指每训练1个epoch都会保存一次权重
    # max_keep_ckpts=5,保留最后几个权重
    # 根据验证集上的验证结果始终保存精度最高的模型
    # checkpoint=dict(type='CheckpointHook', interval=1, max_keep_ckpts=5, save_best='auto'),
    checkpoint=dict(type='CheckpointHook', interval=1),  

    # set sampler seed in distributed evrionment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

# configure environment
env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends)

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

# Defaults to use random seed and disable `deterministic`
randomness = dict(seed=None, deterministic=False)





load_from = 'https://download.openmmlab.com/mmclassification/v0/resnet/resnetv1d50_b32x8_imagenet_20210531-db14775a.pth'
resume = False
randomness = dict(seed=None, deterministic=False)
launcher = 'none'
work_dir = './work_dirs/resnetv1d50_8xb32_in1k'
 # './work_dirs/convmixer-768-32_10xb64_in1k'
"""

with open('/content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py', 'w') as f: 
    f.write(cfg)

# 开始训练

In [38]:
!python /content/mmpretrain/tools/train.py /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py

06/08 02:06:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 989048906
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,

## 推理测试

In [41]:
# !python /content/mmpretrain/tools/train.py /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py
# !python /content/mmpretrain/tools/test.py /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py
!python /content/mmpretrain/tools/test.py /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/epoch_99.pth

06/08 03:42:24 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 988741800
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,

In [45]:
!python /content/mmpretrain/demo/image_demo.py /content/mmpretrain/data/test/柠檬/114.jpg \
/content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/resnetv1d50_8xb32_in1k.py \
--checkpoint /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/epoch_99.pth --show --show-dir ./

Loads checkpoint by local backend from path: /content/mmpretrain/work_dirs/resnetv1d50_8xb32_in1k/epoch_99.pth
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
06/08 04:30:45 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 26592 (\N{CJK UNIFIED IDEOGRAPH-67E0}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 27308 (\N{CJK UNIFIED IDEOGRAPH-6AAC}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 26592 (\N{CJK UNIFIED IDEOGRAPH-67E0}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/utils.py:240: UserWarning: G